In [1]:
import os
import sys
print(sys.executable)

d:\interview projects\optimal solutions\hospital-pricing\myenv\Scripts\python.exe


In [2]:
import sys
!"{sys.executable}" -m pip install pandas

In [2]:
# Import necessary libraries
import pandas as pd
from pathlib import Path

# Path to your raw data directory
DATA_RAW = Path("../data/raw")

In [4]:
hospitals = os.listdir(DATA_RAW)
hospitals

['howard-county-general-hospital_standardcharges.csv',
 'johns-hopkins-hospital_standardcharges.csv',
 'mayo-clinic-arizona_standardcharges.csv',
 'mayo-clinic-florida_standardcharges.csv',
 'mount-sinai-brooklyn_standardcharges.csv',
 'mount-sinai-hospital_standardcharges.csv',
 'mount-sinai-morningside_standardcharges.csv',
 'mount-sinai-queens_standardcharges.csv']

Step 1: Define the Unified Schema

Establish a common structure that covers the key comparable fields across both hospitals.

- Field Name	         Description
- hospital_name          Name of the hospital (e.g., Mount Sinai, Johns Hopkins)
- state                  State the hospital is located in
- county                 County (if retrievable from metadata or external lookup)
- procedure_name         Name/description of the medical procedure
- code                   CPT/HCPCS/DRG/Revenue code
- code_type              The type of medical code (e.g., CPT, HCPCS, etc.)
- setting                Inpatient / Outpatient
- cash_price             Standard cash rate
- cash_discount          Discounted cash rate (if provided)
- payer                  Insurance payer name
- plan                   Specific plan name (optional)
- allowed_amount         Negotiated rate (dollar value)
- min_negotiated_price   Minimum negotiated price across payers
- max_negotiated_price   Maximum negotiated price across payers
- rate_type              How rate is calculated (methodology)

# mount-sinai-brooklyn_standardcharges.csv

In [144]:
df_cols_name = [
    "hospital_code","procedure_name","code","code_type","revenue_code","charge_type","modifier",
    "coding_system","setting","department","sub_department","cash_price","cash_discount","payer",
    "plan","payer_contract_type","payer_rate","min_negotiated_price","max_negotiated_price",
    "allowed_amount","standard_charge","rate_type","notes","revenue_code_override","additional_info"
]

In [145]:
df = pd.read_csv(
    DATA_RAW / "mount-sinai-brooklyn_standardcharges.csv",
    skiprows=3,         # skip first row if it's metadata
    delimiter=',',      # adjust if tab '\t' or semicolon ';'
    on_bad_lines="skip",  # skip malformed lines
    engine='python',     # more flexible parser
    names=df_cols_name,
    nrows=100000
)

In [146]:
df[["hospital_code", "procedure_name", "code", "code_type", "revenue_code", "charge_type", "payer_rate", "allowed_amount"]].head(10)

,hospital_code,procedure_name,code,code_type,revenue_code,charge_type,payer_rate,allowed_amount
0,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,50121.25,999999999
1,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,112772.81,999999999
2,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,112772.81,999999999
3,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,112772.81,999999999
4,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,50121.25,999999999
5,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,52627.31,999999999
6,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,112772.81,999999999
7,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,112772.81,999999999
8,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,51624.89,999999999
9,MSBK,Liver Transplant,0011,APR-DRG,NaN,NaN,60345.98,999999999


Hospital Name

In [147]:
df['hospital_code'].unique()

array(['MSBK'], dtype=object)

The hospital name will be Mount Sinai Brooklyn Since it is only one.

State: New York

In [148]:
for col in ['procedure_name','code','code_type','setting','cash_price','cash_discount','payer','plan','allowed_amount','min_negotiated_price','max_negotiated_price','rate_type']:
    print(f"Unique values in {col}: {df[col].unique()}")
    print(f"Count of values in {col}: {df[col].count()}")
    print(f"Value counts in {col}: {df[col].value_counts()}")
    print("\n")


Unique values in procedure_name: ['Liver Transplant' 'Liver transplant &/or intestinal transplant'
 'Other nervous system & related procedures' ...
 'BIT DRILL OD6 MM SPINE SHAFT DISPOSABLE'
 'SIZER GRAFT ULNAR COLLATERAL LIGAMENT BLOCK'
 'BRONCHOSCOPE VIDEO GLIDESCOPE BFLEX OD5 MM DISPOSABLE']
Count of values in procedure_name: 100000
Value counts in procedure_name: procedure_name
Dial Oth/Thn Hemo 1 Phys Eval - 90945                                                                                             969
Hemo Px W/1 Phys Eval - 90935                                                                                                     461
HC Iopamidol-300 61% 100ml Bu1ml                                                                                                  414
HC Gadobutrol 10mm/10ml Bu0.1ml                                                                                                   345
Tx Swlng Dysf&/Oral Funcj Feeding - 92526                                      

Procedure Name

In [85]:
df['procedure_name'].unique()

array(['Liver Transplant', 'Liver transplant &/or intestinal transplant',
       'Other nervous system & related procedures', ...,
       'BIT DRILL OD6 MM SPINE SHAFT DISPOSABLE',
       'SIZER GRAFT ULNAR COLLATERAL LIGAMENT BLOCK',
       'BRONCHOSCOPE VIDEO GLIDESCOPE BFLEX OD5 MM DISPOSABLE'],
      shape=(5821,), dtype=object)

In [88]:
df[df['procedure_name'].str.contains("liver", case=False)]['procedure_name'].unique()

array(['Liver Transplant', 'Liver transplant &/or intestinal transplant',
       'Major pancreas, liver & shunt procedures',
       'Major Pancreas Liver & Shunt Procedures',
       'Hepatic coma & other major acute liver disorders',
       'Hepatic Coma & Other Major Acute Liver Disorders',
       'Alcoholic liver disease', 'Alcoholic Liver Disease',
       'Other Disorders Of The Liver', 'Other disorders of the liver',
       'Cesarean Delivery', 'Vaginal Delivery w/ Sterilization &/or D&C',
       'Vaginal Delivery w/ Complicating Procedures Exc Sterilization &/or D&C',
       'Vaginal delivery w complicating procedures exc sterilization &/or D&C',
       'Other O.R. proc for obstetric diagnoses except delivery diagnoses',
       'Other O.R. Proc For Obstetric Diagnoses Except Delivery Diagnoses',
       'Vaginal Delivery', 'Cesarean delivery', 'HC Liver Acquisition',
       'HC Liver Acquisition Cadaver', 'HC Liver Acquistn Livng Donor',
       'HC Liver Global Evaluation',
       

In [91]:
df[(df['procedure_name'] == 'Major pancreas, liver & shunt procedures') | (df['procedure_name'] == 'Major Pancreas Liver & Shunt Procedures')]

,hospital_code,procedure_name,code,code_type,revenue_code,charge_type,modifier,coding_system,setting,department,...,payer_contract_type,payer_rate,min_negotiated_price,max_negotiated_price,allowed_amount,standard_charge,rate_type,notes,revenue_code_override,additional_info
9058,MSBK,"Major pancreas, liver & shunt procedures",2601,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,26710.65,NaN,NaN,999999999,11633.97,26710.65,case rate,NaN,NaN
9059,MSBK,"Major pancreas, liver & shunt procedures",2601,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,19586.47,NaN,NaN,999999999,11633.97,26710.65,case rate,NaN,NaN
9060,MSBK,"Major pancreas, liver & shunt procedures",2601,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,12464.97,NaN,NaN,999999999,11633.97,26710.65,case rate,NaN,NaN
9061,MSBK,"Major pancreas, liver & shunt procedures",2601,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,26710.65,NaN,NaN,999999999,11633.97,26710.65,case rate,NaN,NaN
9062,MSBK,"Major pancreas, liver & shunt procedures",2601,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,26710.65,NaN,NaN,999999999,11633.97,26710.65,case rate,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9220,MSBK,Major Pancreas Liver & Shunt Procedures,2604,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,135886.18,NaN,NaN,999999999,59185.98,135886.18,case rate,NaN,NaN
9221,MSBK,Major Pancreas Liver & Shunt Procedures,2604,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,60393.86,NaN,NaN,999999999,59185.98,135886.18,case rate,NaN,NaN
9222,MSBK,Major Pancreas Liver & Shunt Procedures,2604,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,135886.18,NaN,NaN,999999999,59185.98,135886.18,case rate,NaN,NaN
9223,MSBK,Major Pancreas Liver & Shunt Procedures,2604,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,135886.18,NaN,NaN,999999999,59185.98,135886.18,case rate,NaN,NaN


# mount-sinai-hospital_standardcharges.csv

In [119]:
df = pd.read_csv(
    DATA_RAW / "mount-sinai-hospital_standardcharges.csv",
    skiprows=3,         # skip first row if it's metadata
    delimiter=',',      # adjust if tab '\t' or semicolon ';'
    on_bad_lines="skip",  # skip malformed lines
    engine='python',     # more flexible parser
    names=df_cols_name,
    nrows=100000
)

In [120]:
df.head(10)

,hospital_code,procedure_name,code,code_type,revenue_code,charge_type,modifier,coding_system,setting,department,...,payer_contract_type,payer_rate,min_negotiated_price,max_negotiated_price,allowed_amount,standard_charge,rate_type,notes,revenue_code_override,additional_info
0,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,111262.37,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
1,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,40765.05,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
2,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,183580.07,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
3,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,49449.94,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
4,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,49449.94,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
5,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,111262.37,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
6,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,77735.31,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
7,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,111262.37,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
8,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,49449.94,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN
9,MSH,Liver Transplant,0011,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,111262.37,NaN,NaN,999999999,38823.86,209805.79,case rate,NaN,NaN


In [121]:
df[['procedure_name','code','code_type']].head(10)

,procedure_name,code,code_type
0,Liver Transplant,0011,APR-DRG
1,Liver Transplant,0011,APR-DRG
2,Liver Transplant,0011,APR-DRG
3,Liver Transplant,0011,APR-DRG
4,Liver Transplant,0011,APR-DRG
5,Liver Transplant,0011,APR-DRG
6,Liver Transplant,0011,APR-DRG
7,Liver Transplant,0011,APR-DRG
8,Liver Transplant,0011,APR-DRG
9,Liver Transplant,0011,APR-DRG


Hospital name:

In [122]:
df['hospital_code'].unique()

array(['MSH'], dtype=object)

State is New York

In [123]:
df['procedure_name'].unique()

array(['Liver Transplant', 'Liver transplant &/or intestinal transplant',
       'Heart &/or Lung Transplant', ..., 'BIT DRILL 4.5MM',
       'TOOL DISSECTING 5CM SHORT STRAIGHT DIAMOND BALL 2.5MM MICROMAX DISPOSABLE BURR',
       'INSERTER SURGICAL GRAFTBOLT L30 MM OD10 MM'],
      shape=(5909,), dtype=object)

In [125]:
df['code'].unique()
df['code'].count()

np.int64(100000)

In [126]:
df['code_type'].value_counts()

code_type
CDM        73120
APR-DRG    26880
Name: count, dtype: int64

In [128]:
for col in ['procedure_name','code','code_type','setting','cash_price','cash_discount','payer','plan','allowed_amount','min_negotiated_price','max_negotiated_price','rate_type']:
    print(f"Unique values in {col}: {df[col].unique()}")
    print(f"Count of values in {col}: {df[col].count()}")
    print(f"Value counts in {col}: {df[col].value_counts()}")
    print("\n")


Unique values in procedure_name: ['Liver Transplant' 'Liver transplant &/or intestinal transplant'
 'Heart &/or Lung Transplant' ... 'BIT DRILL 4.5MM'
 'TOOL DISSECTING 5CM SHORT STRAIGHT DIAMOND BALL 2.5MM MICROMAX DISPOSABLE BURR'
 'INSERTER SURGICAL GRAFTBOLT L30 MM OD10 MM']
Count of values in procedure_name: 100000
Value counts in procedure_name: procedure_name
Dial Oth/Thn Hemo 1 Phys Eval - 90945                            768
HC Iopamidol-300 61% 100ml Bu1ml                                 378
Hemo Px W/1 Phys Eval - 90935                                    334
HC Gadobutrol 10mm/10ml Bu0.1ml                                  315
Tx Swlng Dysf&/Oral Funcj Feeding - 92526                        275
                                                                ... 
MILL BONE BASE BM110 MIDAS REX                                     1
BIT GRAY PT DRILL 3.2MMX200MM                                      1
ELECTRODE PMT SEEG DEPTH S 12 CONTACT 40.5MM RECORDING LENGTH      1
GUIDE DRIL

# It works the same as Brooklyn Hospital

# mount-sinai-morningside_standardcharges.csv

In [132]:
df = pd.read_csv(
    DATA_RAW / "mount-sinai-morningside_standardcharges.csv",
    skiprows=3,         # skip first row if it's metadata
    delimiter=',',      # adjust if tab '\t' or semicolon ';'
    on_bad_lines="skip",  # skip malformed lines
    engine='python',     # more flexible parser
    names=df_cols_name,
    nrows=100000
)

In [133]:
df.head()

,hospital_code,procedure_name,code,code_type,revenue_code,charge_type,modifier,coding_system,setting,department,...,payer_contract_type,payer_rate,min_negotiated_price,max_negotiated_price,allowed_amount,standard_charge,rate_type,notes,revenue_code_override,additional_info
0,MSSM,Other nervous system & related procedures,0264,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,117013.64,NaN,NaN,999999999,41036.93,117013.64,case rate,NaN,NaN
1,MSSM,Other nervous system & related procedures,0264,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,117013.64,NaN,NaN,999999999,41036.93,117013.64,case rate,NaN,NaN
2,MSSM,Other nervous system & related procedures,0264,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,41036.93,NaN,NaN,999999999,41036.93,117013.64,case rate,NaN,NaN
3,MSSM,Other nervous system & related procedures,0264,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,117013.64,NaN,NaN,999999999,41036.93,117013.64,case rate,NaN,NaN
4,MSSM,Other nervous system & related procedures,0264,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,52006.06,NaN,NaN,999999999,41036.93,117013.64,case rate,NaN,NaN


In [134]:
for col in ['procedure_name','code','code_type','setting','cash_price','cash_discount','payer','plan','allowed_amount','min_negotiated_price','max_negotiated_price','rate_type']:
    print(f"Unique values in {col}: {df[col].unique()}")
    print(f"Count of values in {col}: {df[col].count()}")
    print(f"Value counts in {col}: {df[col].value_counts()}")
    print("\n")


Unique values in procedure_name: ['Other nervous system & related procedures' 'Spinal disorders & injuries'
 'Liver Transplant' ... 'REAMER UNIVERS REVERS GLENOID PREPMED'
 'SHAFT SCREWDRIVER 3.5 MM HEXAGON L19.6 CM OD4 MM TRANSTIBIAL CANNULATED RATCHET HANDLE 3.5 MM SCREW'
 'HANDLE DRIVER IBALANCE HIGH TIBIAL OSTEOTOMY']
Count of values in procedure_name: 100000
Value counts in procedure_name: procedure_name
Dial Oth/Thn Hemo 1 Phys Eval - 90945                                                                                                                     763
HC Iopamidol-300 61% 100ml Bu1ml                                                                                                                          414
Hemo Px W/1 Phys Eval - 90935                                                                                                                             358
HC Gadobutrol 10mm/10ml Bu0.1ml                                                                                  

# It works the same as Brooklyn Hospital

# mount-sinai-queens_standardcharges.csv

In [135]:
df = pd.read_csv(
    DATA_RAW / "mount-sinai-queens_standardcharges.csv",
    skiprows=3,         # skip first row if it's metadata
    delimiter=',',      # adjust if tab '\t' or semicolon ';'
    on_bad_lines="skip",  # skip malformed lines
    engine='python',     # more flexible parser
    names=df_cols_name,
    nrows=100000
)

In [136]:
df.head()

,hospital_code,procedure_name,code,code_type,revenue_code,charge_type,modifier,coding_system,setting,department,...,payer_contract_type,payer_rate,min_negotiated_price,max_negotiated_price,allowed_amount,standard_charge,rate_type,notes,revenue_code_override,additional_info
0,MSQ,Extracranial vascular procedures,0243,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,26732.83,NaN,NaN,999999999,22781.02,66618.78,case rate,NaN,NaN
1,MSQ,Extracranial vascular procedures,0243,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,54394.67,NaN,NaN,999999999,22781.02,66618.78,case rate,NaN,NaN
2,MSQ,Extracranial vascular procedures,0243,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,41842.68,NaN,NaN,999999999,22781.02,66618.78,case rate,NaN,NaN
3,MSQ,Extracranial vascular procedures,0243,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,66618.78,NaN,NaN,999999999,22781.02,66618.78,case rate,NaN,NaN
4,MSQ,Extracranial vascular procedures,0243,APR-DRG,NaN,NaN,NaN,NaN,inpatient,NaN,...,NaN,45016.27,NaN,NaN,999999999,22781.02,66618.78,case rate,NaN,NaN


In [137]:
for col in ['procedure_name','code','code_type','setting','cash_price','cash_discount','payer','plan','allowed_amount','min_negotiated_price','max_negotiated_price','rate_type']:
    print(f"Unique values in {col}: {df[col].unique()}")
    print(f"Count of values in {col}: {df[col].count()}")
    print(f"Value counts in {col}: {df[col].value_counts()}")
    print("\n")


Unique values in procedure_name: ['Extracranial vascular procedures' 'Liver Transplant'
 'Liver transplant &/or intestinal transplant' ...
 'BIT DRILL OD2.8 MM 3 MM KNOTLESS SUTURETAK ANCHOR'
 'TUBING MEDICAL 6FT 3/8IN 3/32IN CARMEDA PRECUT STERILE'
 'BIT DRILL CANNULATED MINI TIGHTROPE']
Count of values in procedure_name: 100000
Value counts in procedure_name: procedure_name
Dial Oth/Thn Hemo 1 Phys Eval - 90945                     763
HC Iopamidol-300 61% 100ml Bu1ml                          372
Hemo Px W/1 Phys Eval - 90935                             332
HC Gadobutrol 10mm/10ml Bu0.1ml                           310
Tx Swlng Dysf&/Oral Funcj Feeding - 92526                 270
                                                         ... 
HOOK RTRCT NICO BRAINPATH SHPRD                             1
GUIDE DRL 1.3MM FPS PILT LCK 2MM SCR                        1
BIT DRILL OD2.8 MM 3 MM KNOTLESS SUTURETAK ANCHOR           1
TUBING MEDICAL 6FT 3/8IN 3/32IN CARMEDA PRECUT STERILE      1
B

# It works the same as Brooklyn Hospital

# johns-hopkins-hospital_standardcharges.csv

In [138]:
with open(DATA_RAW / "johns-hopkins-hospital_standardcharges.csv") as f:
    for _ in range(50):
        print(f.readline())

ï»¿"hospital_name","last_updated_on","version","hospital_location","hospital_address","license_number|MD","To the best of its knowledge and belief, the hospital has included all applicable standard charge information in accordance with the requirements of 45 CFR 180.50, and the information encoded is true, accurate, and complete as of the date indicated.",,,,,,,,,,,,,,,,,

"The Johns Hopkins Hospital","2025-07-01","2.2.0","The Johns Hopkins Hospital","600 North  Wolfe Street, Baltimore, MD 21287","30034","true",,,,,,,,,,,,,,,,,

"description","code|1","code|1|type","code|2","code|2|type","code|3","code|3|type","billing_class","setting","drug_unit_of_measurement","drug_type_of_measurement","modifiers","standard_charge|gross","standard_charge|discounted_cash","payer_name","plan_name","standard_charge|negotiated_dollar","standard_charge|negotiated_percentage","standard_charge|negotiated_algorithm","estimated_amount","standard_charge|methodology","standard_charge|min","standard_charge|max"

In [8]:
df_cols_name = [
    "procedure_name",                 # description
    "code",                           # code|1
    "code_type",                      # code|1|type
    "code_2",                         # code|2
    "code_2_type",                    # code|2|type
    "code_3",                         # code|3
    "code_3_type",                    # code|3|type
    "billing_class",                  # billing_class
    "setting",                        # setting (inpatient/outpatient)
    "drug_unit_of_measurement",       # drug_unit_of_measurement
    "drug_type_of_measurement",       # drug_type_of_measurement
    "modifier",                       # modifiers
    "cash_price",                     # standard_charge|gross
    "cash_discount",                  # standard_charge|discounted_cash
    "payer",                          # payer_name
    "plan",                           # plan_name
    "allowed_amount",                 # standard_charge|negotiated_dollar
    "allowed_percentage",             # standard_charge|negotiated_percentage
    "negotiated_algorithm",           # standard_charge|negotiated_algorithm
    "estimated_amount",               # estimated_amount
    "rate_type",                      # standard_charge|methodology
    "min_negotiated_price",           # standard_charge|min
    "max_negotiated_price",           # standard_charge|max
    "additional_info"                 # additional_generic_notes
]

In [9]:
df = pd.read_csv(
    DATA_RAW / "johns-hopkins-hospital_standardcharges.csv",
    skiprows=3,         # skip first row if it's metadata
    delimiter=',',      # adjust if tab '\t' or semicolon ';'
    on_bad_lines="skip",  # skip malformed lines
    engine='python',     # more flexible parser
    names=df_cols_name,
    nrows=100000
)

In [10]:
df.head()

,procedure_name,code,code_type,code_2,code_2_type,code_3,code_3_type,billing_class,setting,drug_unit_of_measurement,...,payer,plan,allowed_amount,allowed_percentage,negotiated_algorithm,estimated_amount,rate_type,min_negotiated_price,max_negotiated_price,additional_info
0,New Technology - Level 22 ($2001-$2500),1522,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Meritain Health,NaN,NaN,Conditional payment logic at the claim level i...,8051.96,other,2659.27,17544.82,Contracting method is an algorithm described i...
1,New Technology - Level 22 ($2001-$2500),1522,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Health Network Only,NaN,NaN,Conditional payment logic at the claim level i...,3223.68,other,2659.27,17544.82,Contracting method is an algorithm described i...
2,New Technology - Level 22 ($2001-$2500),1522,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Choice Pos I And Ii,NaN,NaN,Conditional payment logic at the claim level i...,8189.59,other,2659.27,17544.82,Contracting method is an algorithm described i...
3,New Technology - Level 22 ($2001-$2500),1522,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna Med Adv,Aetna Medicare Ppo,NaN,NaN,Conditional payment logic at the claim level i...,7554.00,other,2659.27,17544.82,Contracting method is an algorithm described i...
4,New Technology - Level 22 ($2001-$2500),1522,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Blue Cr/Sh Fep,Blue Cr/Sh Fep Focus Ppo,NaN,NaN,Conditional payment logic at the claim level i...,8122.30,other,2659.27,17544.82,Contracting method is an algorithm described i...


Hospital Name: Johns Hopkins Howard County

State MaryLand

In [141]:
for col in ['procedure_name','code','code_type','setting','cash_price','cash_discount','payer','plan','allowed_amount','min_negotiated_price','max_negotiated_price','rate_type']:
    print(f"Unique values in {col}: {df[col].unique()}")
    print(f"Count of values in {col}: {df[col].count()}")
    print(f"Value counts in {col}: {df[col].value_counts()}")
    print("\n")


Unique values in procedure_name: ['New Technology - Level 22 ($2001-$2500)'
 'New Technology - Level 39 ($15,001-$20,000)'
 'Radium ra223 dichloride ther' ... 'Distinct Procedural Service'
 'Modifier description not available'
 'Medicaid level of care 9, as defined by each state']
Count of values in procedure_name: 31222
Value counts in procedure_name: procedure_name
Prolonged Evaluation and Management Services                    146
Medicaid level of care 9, as defined by each state              146
Distinct Procedural Service                                     146
Modifier description not available                              146
Bilateral Procedure                                             146
                                                               ... 
IRB ETCTN10204 NIVOLUMAB INFUSION                                 1
IRB 00268564 ESCITALOPRAM 5 MG OR PLACEBO TABLET                  1
IRB J1994 KRAS PEPTIDE VACCINE WITH POLY-ICLC SUBQ INJECTION      1
IRB J1994 NIVOLUMA

In [60]:
df[['procedure_name','code','code_type']].iloc[800:810]

,procedure_name,code,code_type
800,5071,APC,NaN
801,5071,APC,NaN
802,5071,APC,NaN
803,5071,APC,NaN
804,5071,APC,NaN
805,5071,APC,NaN
806,5071,APC,NaN
807,5071,APC,NaN
808,5071,APC,NaN
809,5071,APC,NaN


In [62]:
df[['procedure_name','code','code_type', 'payer_rate', 'allowed_amount']].iloc[800:810]

,procedure_name,code,code_type,payer_rate,allowed_amount
800,5071,APC,NaN,NaN,1758.75
801,5071,APC,NaN,NaN,1898.92
802,5071,APC,NaN,NaN,11344.55
803,5071,APC,NaN,NaN,2492.12
804,5071,APC,NaN,NaN,931.38
805,5071,APC,NaN,NaN,1419.78
806,5071,APC,NaN,NaN,1355.32
807,5071,APC,NaN,NaN,2196.60
808,5071,APC,NaN,NaN,1589.52
809,5071,APC,NaN,NaN,959.57


In [63]:
df[df['payer_rate'].notna()]

,hospital_code,procedure_name,code,code_type,revenue_code,charge_type,modifier,coding_system,setting,department,...,payer_contract_type,payer_rate,min_negotiated_price,max_negotiated_price,allowed_amount,standard_charge,rate_type,notes,revenue_code_override,additional_info


# howard-county-general-hospital_standardcharges.csv

In [3]:
with open(DATA_RAW / "howard-county-general-hospital_standardcharges.csv") as f:
    for _ in range(50):
        print(f.readline())

ï»¿"hospital_name","last_updated_on","version","hospital_location","hospital_address","license_number|MD","To the best of its knowledge and belief, the hospital has included all applicable standard charge information in accordance with the requirements of 45 CFR 180.50, and the information encoded is true, accurate, and complete as of the date indicated.",,,,,,,,,,,,,,,,,

"Johns Hopkins Howard County Medical Center","2025-07-01","2.2.0","Johns Hopkins Howard County Medical Center","5755 Cedar Lane, Columbia, MD 21044","13-004","true",,,,,,,,,,,,,,,,,

"description","code|1","code|1|type","code|2","code|2|type","code|3","code|3|type","billing_class","setting","drug_unit_of_measurement","drug_type_of_measurement","modifiers","standard_charge|gross","standard_charge|discounted_cash","payer_name","plan_name","standard_charge|negotiated_dollar","standard_charge|negotiated_percentage","standard_charge|negotiated_algorithm","estimated_amount","standard_charge|methodology","standard_charge|mi

In [6]:
df = pd.read_csv(
    DATA_RAW / "howard-county-general-hospital_standardcharges.csv",
    skiprows=3,         # skip first row if it's metadata
    delimiter=',',      # adjust if tab '\t' or semicolon ';'
    on_bad_lines="skip",  # skip malformed lines
    engine='python',     # more flexible parser
    names=df_cols_name,
    nrows=100000
)

In [7]:
df.head()

,procedure_name,code,code_type,code_2,code_2_type,code_3,code_3_type,billing_class,setting,drug_unit_of_measurement,...,payer,plan,allowed_amount,allowed_percentage,negotiated_algorithm,estimated_amount,rate_type,min_negotiated_price,max_negotiated_price,additional_info
0,Level 1 Type A ED Visits,5021,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Hlth Network Option,NaN,NaN,Conditional payment logic at the claim level i...,127.31,other,20.25,2706.38,Contracting method is an algorithm described i...
1,Level 1 Type A ED Visits,5021,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Hmo,NaN,NaN,Conditional payment logic at the claim level i...,139.50,other,20.25,2706.38,Contracting method is an algorithm described i...
2,Level 1 Type A ED Visits,5021,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Managed Choice Open Access,NaN,NaN,Conditional payment logic at the claim level i...,297.49,other,20.25,2706.38,Contracting method is an algorithm described i...
3,Level 1 Type A ED Visits,5021,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Health Network Only,NaN,NaN,Conditional payment logic at the claim level i...,95.36,other,20.25,2706.38,Contracting method is an algorithm described i...
4,Level 1 Type A ED Visits,5021,APC,NaN,NaN,NaN,NaN,facility,outpatient,NaN,...,Aetna,Aetna Choice Pos I And Ii,NaN,NaN,Conditional payment logic at the claim level i...,140.91,other,20.25,2706.38,Contracting method is an algorithm described i...


In [12]:
df[df['code'] == 'code|1'].head()

,procedure_name,code,code_type,code_2,code_2_type,code_3,code_3_type,billing_class,setting,drug_unit_of_measurement,...,payer,plan,allowed_amount,allowed_percentage,negotiated_algorithm,estimated_amount,rate_type,min_negotiated_price,max_negotiated_price,additional_info
